<h2 align="center">Лабораторна робота №2</h2>

#### **Мета роботи:**
Навчитись застосовувати симплекс-метод для вирішення оптимізаційних задач.



#### **Завдання роботи:**
Визначити максимально ефективний розв'язок за відповідних умов обмежень.

#### **Завдання варіанту №1**

$z = 2x_1 - x_2 + 3x_3 - 2x_4 + x_5 (max)$

$
\begin{cases} 
    -1x_1 + 1x_2 + 1x_3 + 0x_4 + 0x_5 = 1\\ 
    \hspace{0.3cm} 1x_1 - 1x_2 + 0x_3 + 1x_4 + 0x_5 = 1\\
    \hspace{0.3cm} 1x_1 + 1x_2 + 0x_3 + 0x_4 + 1x_5 = 2
\end{cases}
$

$C = 
\begin{bmatrix}
    2 & -1 & 3 & -2 & 1
\end{bmatrix}
$

$A = 
\begin{bmatrix}
    -1 &  \hspace{0.3cm}1 & 1 & 0 & 0\\
     \hspace{0.3cm}1 & -1 & 0 & 1 & 0\\
     \hspace{0.3cm}1 &  \hspace{0.3cm}1 & 0 & 0 & 1
\end{bmatrix}
$

$b = 
\begin{bmatrix}
    1 & 1 & 2
\end{bmatrix}^T
$

In [22]:
import numpy as np
import copy

C = np.array([2, -1, 3, -2, 1])
A = np.array([[-1,  1, 1, 0, 0], 
              [ 1, -1, 0, 1, 0], 
              [ 1,  1, 0, 0, 1]])
b = np.array([1, 1, 2])

In [23]:
A[..., 2]

array([1, 0, 0])

In [24]:
unique, counts = np.unique(A[...,3], return_counts=True)
# print((unique == [0, 1]).all())
# print(counts[1] == 1)
# a = np.where(A[..., 4]==1)
# print(a)
for i in A.T:
    print(i)
A.shape

[-1  1  1]
[ 1 -1  1]
[1 0 0]
[0 1 0]
[0 0 1]


(3, 5)

In [25]:
def find_basis(A: np.ndarray) -> np.ndarray:        # повертає список елементів які складають базис
    res = np.zeros(A.shape[0])
    for i in range(A.shape[1]):
        unique, counts = np.unique(A[..., i], return_counts=True)
        if (unique == [0, 1]).all() and counts[1] == 1:
            index = np.where(A[..., i]==1)[0][0]
            res[index] = i
    return np.array(res).astype(int)

In [26]:
find_basis(A)

array([2, 3, 4])

In [27]:
C[find_basis(A)]

array([ 3, -2,  1])

In [28]:
def get_delta(A: np.ndarray, Cb: np.ndarray, c: np.ndarray) -> np.ndarray:
    delta = []
    for i in range(A.shape[1]):
        delta.append(np.inner(A[..., i], Cb) - c[i])
    return np.array(delta)

In [29]:
delta = get_delta(A, C[find_basis(A)], C)
print(delta)
np.where(delta == delta.min())[0][0]

[-6  7  0  0  0]


np.int64(0)

In [30]:
def get_Xb_Xr(Xb: np.ndarray, Xr: np.ndarray) -> np.ndarray:
    res = []
    for i in range(Xb.size):
        res.append(Xb[i]/Xr[i])
    return np.array(res)

In [31]:
xxbr = get_Xb_Xr(b, A[..., 0])
xxbr[np.where(xxbr>=0)].min()

np.float64(1.0)

In [32]:
def get_new_el(a11, a12, a21, a22):
    return a11 - (a12*a21)/a22

In [33]:
def get_x_(basis: np.ndarray, Xb: np.ndarray, n: int):
    res = []
    for i in range(n):
        if i in basis:
            res.append(Xb[np.where(basis==i)[0][0]])
        else:
            res.append(0)
    return np.array(res)

In [34]:
A[1, 1]

np.int64(-1)

In [51]:
def simplex(c, A, b):           # працює тільки для канонічної форми
    # start things ^^
    A = A.astype(np.float64)
    basis = find_basis(A)
    Xb = b
    delta = get_delta(A, c[basis], c)
    print(delta)
    old_A = A
    while (delta < 0).any():
        print(f"A:{A}")
        index_c = np.where(delta == delta.min())[0][0]
        Xb_Xr = get_Xb_Xr(Xb, A[..., index_c])
        index_basis = np.where(Xb_Xr == Xb_Xr[np.where(Xb_Xr>=0)].min())[0][0]
        print(f"index_basis:{index_basis}")
        print(f"index_c:{index_c}")
        
        A[..., index_c] = A[..., basis[index_basis]]
        basis[index_basis] = index_c

        
        a = old_A[index_basis, index_c]
        Xb[index_basis] /= a
        A[index_basis, ...] /= a
        print(f"A{A}")
        
        for i in range(Xb.shape[0]):
            if i != index_basis:
                Xb[i] = get_new_el(Xb[i], old_A[i, index_c], Xb[index_basis], a)
            print(f"Xb{old_A[i, index_c]}")
        
        
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                if i!=index_basis and j!=index_c and j not in basis:
                    A[i, j] = get_new_el(old_A[i, j], old_A[i, index_c], old_A[index_basis, j], a)
        
        delta = get_delta(A, c[basis], c)
        print(A)
        print(delta)
        old_A = A
    
    res_x = get_x_(basis, Xb, c.shape[0])
    return res_x, np.inner(res_x, c)

In [52]:
simplex(C, A, b)

[-6.  7.  0.  0.  0.]
A:[[-1.  1.  1.  0.  0.]
 [ 1. -1.  0.  1.  0.]
 [ 1.  1.  0.  0.  1.]]
index_basis:1
index_c:0
A[[ 0.  1.  1.  0.  0.]
 [ 1. -1.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.]]
Xb0.0
Xb1.0
Xb0.0
[[ 0.  1.  1.  0.  0.]
 [ 1. -1.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.]]
[0. 3. 0. 4. 0.]


(array([1, 0, 1, 0, 2]), np.int64(7))